In [ ]:
import tensorflow as tf
import numpy as np
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
np.random.seed(100)

In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input,Activation,GRU
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
from joblib import Parallel,delayed
from scipy.stats import mode

/cerebralcortex/kessel_jupyter_virtualenv/tensorflow/lib/python3.7/site-packages/pandas-1.2.3-py3.7-linux-x86_64.egg/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
directory = './data/10/Walking/'
X,y = [],[]
for i,f in enumerate(list(os.listdir(directory))):
    if f[0]=='.':
        continue
    df  = pickle.load(open(directory+f,'rb'))
    df['time'] = df['window'].apply(lambda a:a[0])
    df['data'] = df['data'].apply(lambda a:np.array([np.array(b) for b in a]))
    df['data'] = df['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
    df['data'] = df['data'].apply(lambda a:a[:,:,1:])
    df = df.sort_values('time').reset_index(drop=True)
    user_id = df.user.values[0]
    X.append(np.concatenate(list(df['data'].values)))
    y.append(user_id)
#     X_test.append(np.concatenate(list(df['data'].values)))
#     y_train.extend([user_id]*X_train[-1].shape[0])
#     y_test.extend([user_id]*X_test[-1].shape[0])
    del df
    print(i,end=',')
pickle.dump([X,y],open('./data/10/walking_all_data1.p','wb'))

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,

In [ ]:
# pickle.dump([X,y],open('./data/10/sitting_all_data1.p','wb'))

In [ ]:
directory = './data/10/Walking/'
data_all = []
for f in list(os.listdir(directory)):
    if f[0]=='.':
        continue
    data_all.append(pickle.load(open(directory+f,'rb')))
data_all[-1].head()  
data  = pd.concat(data_all)
del data_all
unique_users = data.user.unique()
unique_users_dict = {u:i for i,u in enumerate(unique_users)}
data['user_id'] = data['user'].apply(lambda a:unique_users_dict[a])
data['time'] = data['window'].apply(lambda a:a[0])
data['data'] = data['data'].apply(lambda a:np.array([np.array(b) for b in a]))
data['data'] = data['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
data['data'] = data['data'].apply(lambda a:a[:,:,1:])
pickle.dump(data,open('./data/10/walking_all_data.p','wb'))

In [ ]:
X,y = pickle.load(open('./data/10/sitting_all_data.p','rb'))

In [ ]:
def get_model(input_shape=(500,3),n_classes=1):
    model =  Sequential()
    model.add(Conv1D(128,2,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
#     model.add(Conv1D(128,2,activation='relu',kernel_initializer='normal',padding='same'))
#     model.add(MaxPooling1D(2))
    model.add(Activation('tanh'))
#     model.add(BatchNormalization())
    model.add(Dropout(.5))
    model.add(GRU(128,return_sequences=True,activation='tanh'))
    model.add(Flatten())
    model.add(Dense(n_classes))
    model.add(Dense(n_classes,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam')
    return model







def get_test_scores(model,x_test,y_test,test_size=None,n_iter=1000,base_window_size=20):
    def get_score(user,test_size,n_iter,base_window_size):
        df = test_df[test_df.original==user]
        score1 = []
        score2 = []
        test_lengths = []
        for t in test_size:
            n  = np.int64(t*60/base_window_size)
            if n>df.shape[0]:
                continue
            y_true = [df['original'].values[0]]*n_iter
            y_pred_majority = []
    #         y_pred_maxmean = []
            for i in range(n_iter):
                temp_df = df.sample(n,replace=False)
                y_pred_majority.append(mode(temp_df['prediction'].values)[0][0])
    #             temp_df = np.mean(np.concatenate(list(temp_df['probability'])),axis=0)
    #             y_pred_maxmean.append(np.argmax(temp_df))


            majority_score = accuracy_score(y_true,y_pred_majority)
    #         maxmean_score = accuracy_score(y_true,y_pred_maxmean)
            score1.append(majority_score)
    #         score2.append(maxmean_score)
            test_lengths.append(t)
        
        df = pd.DataFrame({'majority_score':score1,
    #                         'maxmean_score':score2,
                            'test_lengths':test_lengths})
#         print(df.loc[0])
        return df
    
    y_pred = model.predict(x_test).argmax(axis=1)
#     y_pred1 = y_pred.argmax(axis=1)
    test_df = pd.DataFrame({'prediction':y_pred,'original':y_test})
#                             'probability':list(y_pred)})
#     test_df['probability'] = test_df['probability'].apply(lambda a:a.reshape(1,-1)) 
    if test_size is None:
        test_size = np.arange(1,120,1)
    del x_test
    
    result  = Parallel(verbose=2,n_jobs=30)(delayed(get_score)(user,test_size,n_iter,base_window_size) for user in np.unique(test_df.original.values))
    test_score = pd.concat(result)
    acc_ = accuracy_score(test_df['original'],test_df['prediction'])
    
    test_score = test_score.append(pd.DataFrame({'majority_score':[acc_],
#                                                     'maxmean_score':[acc_],
                                                    'test_lengths':[0]}))
    test_score = test_score.groupby('test_lengths',as_index=False).mean()
    return test_score



def get_trained_model(X_train,y_train,n_timesteps,n_channels,window_size,filepath):
    n_classes = len(np.unique(y_train))
    model =  get_model(input_shape=(n_timesteps,n_channels),n_classes=n_classes)
#     model.summary()
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=False)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
    callbacks_list = [es,checkpoint]
#     train_x,test_x,train_y,test_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    train_x,val_x,train_y,val_y = train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=200, batch_size=300,verbose=0,
          callbacks=callbacks_list,shuffle=True)
    model.load_weights(filepath)
#     y_pred = model.predict(test_x).argmax(axis=1)

#     from sklearn.metrics import classification_report

#     print(accuracy_score(test_y,y_pred),window_size)
    return model


def get_training_testing(window_size=10,
                         base_window_size=10):
    n_train = window_size*60//base_window_size
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    def split_data(df):
        filepath = directory+df['user'].values[0]
        df  = pickle.load(open(filepath,'rb'))
        if df.shape[0]<n_train+30*6:
            print(filepath[-2:],end=',')
            return 
        df['time'] = df['window'].apply(lambda a:a[0])
        df['data'] = df['data'].apply(lambda a:np.array([np.array(b) for b in a]))
        df['data'] = df['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
        df['data'] = df['data'].apply(lambda a:a[:,:,1:])
        df = df.sort_values('time').reset_index(drop=True)
        user_id = df.user.values[0]
        X_train.append(np.concatenate(list(df[:n_train]['data'].values)))
        X_test.append(np.concatenate(list(df[n_train:]['data'].values)))
        y_train.extend([user_id]*X_train[-1].shape[0])
        y_test.extend([user_id]*X_test[-1].shape[0])
        print(filepath[-2:],end=',')
        return 
    data.groupby('user',as_index=False).apply(split_data)
    return np.concatenate(X_train),np.concatenate(X_test),y_train,y_test

def get_training_testing_sitting(window_size=10,
                         base_window_size=10,n_user=300):
    n_train = window_size*60//base_window_size
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    count = 0
    i = 0
    while i<len(X):
        if X[i].shape[0]<n_train+30*6:
            i+=1
            continue
        X_train.append(X[i][:n_train])
        X_test.append(X[i][n_train:])
        y_train.extend([y[i]]*X_train[-1].shape[0])
        y_test.extend([y[i]]*X_test[-1].shape[0])
        count+=1
        i+=1
        if count==n_user:
            break
    return np.concatenate(X_train),np.concatenate(X_test),y_train,y_test


# directory = './data/walking_10/right_wrist/'
# filepath1 = './data/all_walking_data_10.p'
# if not os.path.isfile(filepath1):
#     print(filepath1)
#     data_col = []
#     for f in os.listdir(directory):
#         filepath = directory+f
#         data  = pickle.load(open(filepath,'rb'))
#         data_col.append(df)
#     data = pd.concat(data_col)
#     unique_users = data.user.unique()
#     unique_users_dict = {u:i for i,u in enumerate(unique_users)}
#     data['user_id'] = data['user'].apply(lambda a:unique_users_dict[a])
#     data['time'] = data['window'].apply(lambda a:a[0])
#     data['data'] = data['data'].apply(lambda a:np.array([np.array(b) for b in a]))
#     data['data'] = data['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
#     data['data'] = data['data'].apply(lambda a:a[:,:,1:])
#     pickle.dump(data,open(filepath1,'wb'))
# data = pd.DataFrame({'user':list(os.listdir(directory))})


def get_test_score(window_size,base_window_size,n_user,model_save_path,n_timesteps,n_channels,test_lengths):
    X_train,X_test,y_train,y_test = get_training_testing_sitting(window_size=window_size,base_window_size=base_window_size,n_user=n_user)
    unique_users = np.unique(y_train+y_test)
    unique_users_dict = {u:i for i,u in enumerate(unique_users)}
    y_train = np.array([unique_users_dict[a] for a in y_train])
    y_test = np.array([unique_users_dict[a] for a in y_test])
    model_save_path1 = model_save_path+ str(window_size)+'/'
    if not os.path.isdir(model_save_path1):
        os.makedirs(model_save_path1)   
    pickle.dump(unique_users_dict,open(model_save_path1+str(n_user)+'user_dict.p','wb'))   
    filepath = model_save_path1+str(n_user)+'persons.hdf5'
    model = get_trained_model(np.nan_to_num(X_train),y_train,n_timesteps,n_channels,window_size,filepath=filepath)
    test_score = get_test_scores(model,np.nan_to_num(X_test),y_test,test_size=test_lengths,n_iter=1000,base_window_size=base_window_size)
    test_score['train_user'] = n_user
    test_score['train_lengths'] = window_size
    del X_train
    del X_test
    del y_train
    del y_test
    return test_score


base_window_size = 10
Fs = 25
n_timesteps = base_window_size*Fs
n_channels = 3
# window_sizes = [10,20,40,50]+list(np.arange(30,330,30))
window_sizes = np.arange(330,700,30)
window_sizes = np.arange(720,1200,30)
test_lengths = [10,20] + list(np.arange(0,330,30)[1:])
n_users = list(range(50,350,50))
model_save_path = './models/'+str(base_window_size)+'/sitting/'
result_save_path = './results/sitting/'
if not os.path.isdir(result_save_path):
    os.makedirs(result_save_path)
if not os.path.isdir(model_save_path):
    os.makedirs(model_save_path)
results = []
for window_size in window_sizes:
    results = []
    for n_user in n_users:
        test_score = get_test_score(window_size,base_window_size,n_user,model_save_path,n_timesteps,n_channels,test_lengths)
        results.append(test_score)
        pickle.dump(pd.concat(results),open(result_save_path+str(window_size)+'.p','wb'))
        print(n_user,end=',')
    print(window_size,'done','-'*20)

In [ ]:
model_paths = ['./models/10/sitting/'+str(i)+'/300persons.hdf5' for i in np.arange(180,720,30)]
user_dicts = ['./models/10/sitting/'+str(i)+'/300user_dict.p' for i in np.arange(180,720,30)]

users_all = [pickle.load(open(user_dict,'rb')) for user_dict in user_dicts]
user_list = [list(a.keys()) for a in users_all]
from functools import reduce
all_users = list(set(list(reduce(lambda a,b:a+b,user_list))))

models = [tf.keras.models.load_model(model_path) for model_path in model_paths]


data_directory = './data/10/Sitting_test/'


def get_user_data(user):
    df = pickle.load(open(data_directory+user+'.p','rb'))
    df['time'] = df['window'].apply(lambda a:a[0])
    df['data'] = df['data'].apply(lambda a:np.array([np.array(b) for b in a]))
    df['data'] = df['data'].apply(lambda a:a[a[:,0].argsort(),:].reshape(1,-1,4))
    df['data'] = df['data'].apply(lambda a:a[:,:,1:])
    df = df.sort_values('time').reset_index(drop=True)
    return df

y_pred = []
y_orig = []
train_lengths = []
for i,user in enumerate(all_users):
    try:
        df = get_user_data(user)
        X_test = np.concatenate(list(df['data']))
    except:
        continue
    for j,train_length in enumerate(np.arange(180,720,30)):
        if user not in users_all[j]:
            continue
        pred = models[j].predict(X_test).argmax(axis=1)
        orig = [users_all[j][user]]*len(pred)
        y_pred.extend(list(pred))
        y_orig.extend(list(orig))
        train_lengths.extend([train_length]*len(orig))
    print(i,end=',')
test_dfs = pd.DataFrame({'prediction':y_pred,'original':y_orig,'train_lengths':train_lengths})
pickle.dump(test_dfs,open('./data/temp.p','wb'))

In [ ]:
test_dfs = pickle.load(open('./data/temp.p','rb'))

def get_score(user,test_size,n_iter,base_window_size):
    df = test_df[test_df.original==user]
    score1 = []
    score2 = []
    test_lengths = []
    for t in test_size:
        n  = np.int64(t*60/base_window_size)
        if n>df.shape[0]:
            continue
        y_true = [df['original'].values[0]]*n_iter
        y_pred_majority = []
#         y_pred_maxmean = []
        for i in range(n_iter):
            temp_df = df.sample(n,replace=False)
            y_pred_majority.append(mode(temp_df['prediction'].values)[0][0])
#             temp_df = np.mean(np.concatenate(list(temp_df['probability'])),axis=0)
#             y_pred_maxmean.append(np.argmax(temp_df))


        majority_score = accuracy_score(y_true,y_pred_majority)
#         maxmean_score = accuracy_score(y_true,y_pred_maxmean)
        score1.append(majority_score)
#         score2.append(maxmean_score)
        test_lengths.append(t)
        
    df = pd.DataFrame({'majority_score':score1,
#                         'maxmean_score':score2,
                        'test_lengths':test_lengths})
#         print(df.loc[0])
    return df


test_size = [10,20] + list(np.arange(0,730,30)[1:])
base_window_size = 10
n_iter = 1000    

results = []
for tr_length in np.arange(180,720,30):
    test_df = test_dfs[test_dfs.train_lengths==tr_length]

    result  = Parallel(verbose=2,n_jobs=30)(delayed(get_score)(user,test_size,n_iter,base_window_size) for user in np.unique(test_df.original.values))
    test_score = pd.concat(result)
    acc_ = accuracy_score(test_df['original'],test_df['prediction'])

    test_score = test_score.append(pd.DataFrame({'majority_score':[acc_],
    #                                                     'maxmean_score':[acc_],
                                                    'test_lengths':[0]}))
    test_score = test_score.groupby('test_lengths',as_index=False).mean()
    test_score['train_lengths'] = tr_length
    results.append(test_score)
    pickle.dump(pd.concat(results),open('./data/sitting_big_result.p','wb'))

In [ ]:
import pickle
data = pickle.load(open('./data/sitting_big_result.p','rb'))
data = data[~data.train_lengths.isin([510,600,690])]

In [ ]:
import pandas as pd
df = pd.pivot_table(data,columns='train_lengths',index='test_lengths',values='majority_score',aggfunc='mean')

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
sns.heatmap(df,annot=True,fmt='.2f',cmap='Reds')
plt.yticks(rotation=0)
plt.show()

In [ ]:
train_lengths = data['train_lengths'].unique()
plt.figure(figsize=(20,20))
for tr_length in train_lengths:
    df = data[data.train_lengths==tr_length]
    plt.plot(df['test_lengths'],df['majority_score'],label=str(tr_length*6))
plt.show()